In [15]:
import pandas as pd
import numpy as np 
import configparser 
import pymysql
from tqdm import tqdm

In [3]:
pwd

'/home/ubuntu/infoedge/llama-recipes/examples'

In [6]:
config = configparser.ConfigParser()

config.read('/home/ubuntu/infoedge/llama-recipes/examples/configs/config.ini')

['/home/ubuntu/infoedge/llama-recipes/examples/configs/config.ini']

In [7]:
 

mydb= pymysql.connect(

host=config['iimjobs_db']['hostname'],

user=config['iimjobs_db']['username'],

password=config['iimjobs_db']['password'],

db=config['iimjobs_db']['db_name']

)

 

mycursor = mydb.cursor()

 

In [8]:
query = "SELECT id FROM profile_info WHERE last_active_date > '2023-12-03' ORDER BY last_active_date DESC limit 200"

 

mycursor.execute(query)

result = mycursor.fetchall()

In [10]:
user_df=pd.DataFrame(result,columns=['userid'])

In [27]:
user_df

,id,designation,keywords,user_experience,resume


In [26]:
user_df[['userid']]

KeyError: "None of [Index(['userid'], dtype='object')] are in the [columns]"

In [23]:
# user_df['userid']

In [24]:
uid_list =list(user_df[['userid']])


KeyError: "None of [Index(['userid'], dtype='object')] are in the [columns]"

In [20]:

 
 
def clean_space(text):
    return " ".join(re.split("\s+", text, flags=re.UNICODE))

def remove_extra_spaces(input_string):
    # Use regular expression to remove extra spaces (only spaces)
    cleaned_string = re.sub(r' +', ' ', input_string)
    return cleaned_string

def get_user_data_search_embed(user_id_list):
    
    '''
    This function combines all the functions for fetching and processing 
    user data from elastic search
    '''


    sub_json = fetch_data_es(user_id_list)
    raw_data = construct_user_data_search_embed(sub_json)

    res_df = pd.DataFrame.from_dict(raw_data, orient='index').T
    result_df = res_df.astype(str).replace("None", " ")
    
    result_df['resume'] = result_df['resume'].apply(remove_extra_spaces)
    
    return result_df

def fetch_data_es(user_id_list):
    
    """
    Input : List of Valid Hirist User ID 
    Output : List of User Data JSON stored in Elastic Search
    """
    
    
    response_json = []
    replacements = {'null':'None', 'false':'False'}

    url = "http://10.205.2.137:8080/v1/recruiter/-123/applicant/search/searchById"

    res = []

    for uid in tqdm(user_id_list):


        payload = {
            'usersToSearchFrom' : [str(uid)]
        }

        headers = {
        'Content-Type': 'application/json'
        }

        try:
            response = requests.request("POST", url, headers=headers, data = str(payload))
            response_json = response.json()

            res.append(response_json['docs']) 

        except Exception as e:
            continue
        
    return res



def construct_user_data_search_embed(user_json):
    '''
    This is a function for fetching user ID, Keywords (in the form of tags) and 
    user designation from elastic search. This is specifically made for creating 
    user embeddings for our Recruiter search model/POC
    '''
    
    user_dict = {
        'id' : [],
        'designation' : [],
        'keywords' : [],
        'user_experience':[],
        'resume' : []
    }

    for s in tqdm(user_json):

        try:
            source = s[0]

            user_dict['id'].append(source['id'])
            user_dict['resume'].append(source['resumeText'])
            user_dict['designation'].append(source['currentDesignation'])

            #extract a list of user skills 
            skill_list = [str(skill['name']) for skill in source['skillSet']]
#             user_dict['keywords'].append(" ".join(skill_list))
            user_dict['keywords'].append(skill_list)
            user_dict['user_experience'].append(source['expYear'])

        except Exception as e:
            continue

    return user_dict

In [16]:
user_df = get_user_data_search_embed([2029706])


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 21509.25it/s]
0it [00:00, ?it/s]


In [17]:
user_df

,id,designation,keywords,user_experience,resume
